# Previously On Gradient Boosting

**Limitation on Gradient Boosting**
The Biggest computational bottleneck in scaling gradient boosting to large (with many training examples) or high-dimensional (with many features) data sets is tree learning, specifically, identifying optimal splits in the regression tree base estimators

**Approach 1 : Histogram based Gradient Boosting:**
Histogram-based gradient boosting attempts to address this computational bottleneck. This works reasonably well for `medium-sized data sets`. However, histogram-bin construction can itself be slow if we have a very large number of data points or a large number of features.

# LightGBM

**Approach 2 : LightGBM**
1. LightGBM implements that often lead to significant speedups in training times in practice.
2. **Gradient-based One Side Sampling (GOSS)**: aims to reduce the `number of training examples`.
3. **Exclusive Feature Bundling (EFB)** aims to reduce the `number of features`.

## Gradient-based One Side Sampling (GOSS)

1. A well-known approach to dealing with a very large number of training examples is to **down sample** the data set, that is, randomly sample a smaller subset of the data set.
   * not all examples are equally important; (Adaboost use weights)
   * sampling should also ensure that some fraction of correctly classified examples is also included.(to avoid overfitting)
2. Similar to AdaBoost that uses sample weights, GOSS uses the gradient magnitude.
3. Select the top a% of examples with the largest gradients; call this subset top.
4. Randomly sample b% of the remaining examples; call this subset rand.
5. Assign weights to examples in both sets: w_top = 1 and w_rand = (100-a)/b
6. Train a base regressor over this sampled data: (data, -gradients, w)

## Exclusive Feature Bundling (EFB)

1. when feature space is sparse, and features are mutually exclusive.
2. EFB exploits this sparsity and aims to merge mutually exclusive columns into one column to reduce the number of effective features.
3. Identify features that can be bundled together by `measuring conflicts or the number of times both features are non-zero`. The intuition here is that if two features are often simultaneously zero, they are low conflict and can be bundled together.
4. Merge the identified low conflict features into a feature bundle. The idea here is to preserve information carefully when merging non-zero values, which is typically done by adding offsets to feature values to prevent overlaps

In [9]:
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.use("nbagg")
mpl.rcParams["figure.figsize"] = (4, 3)

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=124)

In [3]:
from lightgbm import LGBMClassifier

gbm = LGBMClassifier(boosting_type="gbdt", n_estimators=20, max_depth=1)
gbm.fit(X_train, y_train)

LGBMClassifier(max_depth=1, n_estimators=20)

In [4]:
from sklearn.metrics import accuracy_score

y_pred = gbm.predict(X_test)
test_error = 1 - accuracy_score(y_test, y_pred)
test_error

0.07692307692307687

## Learning rate
Bby selecting an effective learning rate, we try to control the rate at which the model learns so that it doesn't rapidly fit, and then overfit the training data. We can think of this a proactive modeling approach, where we try to identify a good training strategy so that it leads to a good model.

### Manual

In [5]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

n_learning_rates, n_folds = 10, 10
learning_rates = np.linspace(0.1, 1, num=n_learning_rates)

splitter = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=74)

val_error = np.full(shape=(n_learning_rates, n_folds), fill_value=0.0)
train_error = np.full(shape=(n_learning_rates, n_folds), fill_value=0.0)

In [29]:
for i, lr in enumerate(learning_rates):
    for j, (train_idx, val_idx) in enumerate(splitter.split(X_train, y_train)):
        gbm = LGBMClassifier(boosting_type="gbdt", max_depth=1,
                             n_estimators=10, learning_rate=lr)
        gbm.fit(X[train_idx, :], y[train_idx])
        train_error[i, j] = (1 - accuracy_score(y[train_idx],
                                                gbm.predict(X[train_idx, :]))) * 100
        val_error[i, j] = (1 - accuracy_score(y[val_idx],
                                              gbm.predict(X[val_idx, :]))) * 100

mean_train_error = np.mean(train_error, axis=1)
mean_val_error = np.mean(val_error, axis=1)

In [30]:
plt.plot(learning_rates, mean_train_error, label="train error", marker="s")
plt.plot(learning_rates, mean_val_error, label="val error", marker="s")
plt.xlabel("Learning rate")
plt.ylabel("Error (%)")
plt.legend(loc="lower left")
plt.title("Manual calculation")
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

In [32]:
gbm = LGBMClassifier(boosting_type="gbdt",
                     max_depth=1,
                     learning_rate=0.4,
                     n_estimators=20)
gbm.fit(X_train, y_train)
test_error = 1 - accuracy_score(y_test, gbm.predict(X_test))
test_error

0.034965034965035

### sklearn

In [33]:
def error_score(estimator, X, y):
    return (1 - accuracy_score(y, estimator.predict(X))) * 100

In [34]:
from sklearn.model_selection import validation_curve

train_error, val_error = validation_curve(
    LGBMClassifier(max_depth=1, boosting_type="gbdt", learning_rate=0.1, n_estimators=10),
    X=X_train, y=y_train, cv=n_folds,
    param_name="learning_rate",
    param_range=learning_rates,
    scoring=error_score)

train_error_mean = np.mean(train_error, axis=1)
train_error_std = np.std(train_error, axis=1)
val_error_mean = np.mean(val_error, axis=1)
val_error_std = np.std(val_error, axis=1)

In [35]:
plt.plot(learning_rates, train_error_mean, label="train error", marker="s", c="r", lw=2)
plt.fill_between(learning_rates,
                 train_error_mean - train_error_std,
                 train_error_mean + train_error_std,
                 alpha=0.5, color="r",
                 lw=1)
plt.plot(learning_rates, val_error_mean, label="val error", marker="s", c="b", lw=2)
plt.fill_between(learning_rates,
                 val_error_mean - val_error_std,
                 val_error_mean + val_error_std,
                 alpha=0.5, color="b",
                 lw=1)
plt.xlabel("Learning rate")
plt.ylabel("Error (%)")
plt.legend(loc="upper right")
plt.title("sklearn calculation")
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

In [36]:
gbm = LGBMClassifier(boosting_type="gbdt",
                     max_depth=1,
                     learning_rate=0.5,
                     n_estimators=20)
gbm.fit(X_train, y_train)
test_error = 1 - accuracy_score(y_test, gbm.predict(X_test))
test_error

0.04195804195804198

### LightGBM

In [37]:
from lightgbm import cv, Dataset

train_data = Dataset(data=X_train, label=y_train)
params = dict(boosting_type="gbdt",
              learning_rate=0.25,
              max_depth=1,
              objective="cross_entropy")
cv_results = cv(params,
                train_set=train_data,
                num_boost_round=100,
                nfold=5, stratified=True, shuffle=True)

[LightGBM] [Info] [cross_entropy:Init]: (objective) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: (metric) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: sum-of-weights = 340.000000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001163 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4260
[LightGBM] [Info] Number of data points in the train set: 340, number of used features: 30
[LightGBM] [Info] [cross_entropy:Init]: (metric) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: sum-of-weights = 86.000000
[LightGBM] [Info] [cross_entropy:Init]: (objective) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: (metric) labels passed interval [0, 1] check
[LightGBM] [Info] [cross_entropy:Init]: sum-of-weights = 341.000000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of

In [41]:
plt.plot(np.arange(100), cv_results["cross_entropy-mean"], c="b", lw=2)
plt.fill_between(np.arange(100),
                 np.array(cv_results["cross_entropy-mean"]) - np.array(cv_results["cross_entropy-stdv"]),
                 np.array(cv_results["cross_entropy-mean"]) + np.array(cv_results["cross_entropy-stdv"]),
                 color="b", alpha=0.5, lw=2)

plt.title("Light GBM")
plt.xlabel("Number of boosting iteration")
plt.ylabel("cross entropy")
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

## Early Stopping

 Early stopping with LightGBM works slightly differently than AdaBoost. In LightGBM, we specify a value for the parameter early_stopping_rounds. As long as the overall score (say accuracy) improves over the last `early_stopping_rounds`, LightGBM will continue to train. However, when the score does not improve after early_stopping_rounds, LightGBM stops.

Small values of early_stopping_rounds make LightGBM very "impatient" and aggressive in that it does not wait too long to see if there is any improvement before stopping learning early. This may lead to underfitting. Large values of early_stopping_rounds make LightGBM overly patient and too passive in that it waits for longer periods to see if there is any improvement. This may lead to overfitting.

LightGBM needs us to `explicitly specify a validation set as well as scoring metric`. In the experiment below, we split the training data further into a smaller training set and a separate validation set for early stopping. The metric we will use is accuracy score

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
gbm = LGBMClassifier(boosting_type="gbdt",
                     n_estimators=50,
                     max_depth=1, early_stopping=5)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric="auc")

[LightGBM] [Warning] early_stopping_round is set=5, early_stopping=5 will be ignored. Current value: early_stopping_round=5
[1]	valid_0's auc: 0.885522	valid_0's binary_logloss: 0.602321
[2]	valid_0's auc: 0.961022	valid_0's binary_logloss: 0.542925
[3]	valid_0's auc: 0.96528	valid_0's binary_logloss: 0.497678
[4]	valid_0's auc: 0.989846	valid_0's binary_logloss: 0.451922
[5]	valid_0's auc: 0.986243	valid_0's binary_logloss: 0.421817
[6]	valid_0's auc: 0.985588	valid_0's binary_logloss: 0.393713
[7]	valid_0's auc: 0.989846	valid_0's binary_logloss: 0.364781
[8]	valid_0's auc: 0.990501	valid_0's binary_logloss: 0.338063
[9]	valid_0's auc: 0.989191	valid_0's binary_logloss: 0.319267
[10]	valid_0's auc: 0.990501	valid_0's binary_logloss: 0.299509
[11]	valid_0's auc: 0.989519	valid_0's binary_logloss: 0.286114
[12]	valid_0's auc: 0.989519	valid_0's binary_logloss: 0.269481
[13]	valid_0's auc: 0.992303	valid_0's binary_logloss: 0.255033
[14]	valid_0's auc: 0.993285	valid_0's binary_logloss:

LGBMClassifier(early_stopping=5, max_depth=1, n_estimators=50)

In [49]:
number_of_steps = np.arange(1, 10, 1)

trn_err = np.zeros((len(number_of_steps),))
val_err = np.zeros((len(number_of_steps),))
n_iters = np.zeros((len(number_of_steps),))

for i, rounds in enumerate(number_of_steps):
    gbm = LGBMClassifier(boosting_type='gbdt', n_estimators=50, max_depth=1, early_stopping=rounds)
    gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='auc')

    trn_err[i] = 1 - accuracy_score(y_train, gbm.predict(X_train))
    val_err[i] = 1 - accuracy_score(y_val, gbm.predict(X_val))
    n_iters[i] = len(
        gbm.evals_result_['valid_0']['auc'])  # Get the number of estimators in the ensemble in a roundabout way

[1]	valid_0's auc: 0.885522	valid_0's binary_logloss: 0.602321
[2]	valid_0's auc: 0.961022	valid_0's binary_logloss: 0.542925
[3]	valid_0's auc: 0.96528	valid_0's binary_logloss: 0.497678
[4]	valid_0's auc: 0.989846	valid_0's binary_logloss: 0.451922
[5]	valid_0's auc: 0.986243	valid_0's binary_logloss: 0.421817
[LightGBM] [Warning] early_stopping_round is set=2, early_stopping=2 will be ignored. Current value: early_stopping_round=2
[1]	valid_0's auc: 0.885522	valid_0's binary_logloss: 0.602321
[2]	valid_0's auc: 0.961022	valid_0's binary_logloss: 0.542925
[3]	valid_0's auc: 0.96528	valid_0's binary_logloss: 0.497678
[4]	valid_0's auc: 0.989846	valid_0's binary_logloss: 0.451922
[5]	valid_0's auc: 0.986243	valid_0's binary_logloss: 0.421817
[6]	valid_0's auc: 0.985588	valid_0's binary_logloss: 0.393713
[LightGBM] [Warning] early_stopping_round is set=3, early_stopping=3 will be ignored. Current value: early_stopping_round=3
[1]	valid_0's auc: 0.885522	valid_0's binary_logloss: 0.60232

In [53]:
def get_colors(color_map, n_colors):
    cmap_ = mpl.cm.get_cmap(color_map)
    colors_rgb = cmap_(np.linspace(0, 1, n_colors))
    colors_hex = [mpl.colors.rgb2hex(c_) for c_ in colors_rgb]
    return colors_hex


cm = get_colors(color_map="RdBu", n_colors=2)

In [55]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))

ax[0].plot(number_of_steps, trn_err, marker='o', c=cm[0], markeredgecolor='w', linewidth=2)
ax[0].plot(number_of_steps, val_err, marker='s', c=cm[1], markeredgecolor='w', linewidth=2)
ax[0].legend(['Train err', 'Validation err'])
ax[0].set_xlabel('early_stopping_rounds')
ax[0].set_ylabel('Error (%)')
ax[0].set_xticks(range(10))

ax[1].plot(number_of_steps, n_iters, marker='o', c=cm[0], markeredgecolor='w', linewidth=2)
ax[1].set_xlabel('early_stopping_rounds')
ax[1].set_ylabel('Ensemble size at early stopping')
ax[1].set_xticks(range(10))

fig.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

## Custom Loss Function

1. Most powerful features of gradient boosting is that it is applicable to a wide variety of loss functions.
2. Our data set is imbalanced, meaning that different classes have different amounts of data. In such situations, rather than high accuracy, we might require high recall (fewer false negatives, for example, in medical diagnosis) or high precision (fewer false positives, for example, in spam detection)

### Implementing custom loss function "Focal Loss"

The focal loss was introduced for dense object detection, or the problem of object detection at a large number of densely packed windows in an image. Ultimately, such object detection tasks come down to a foreground vs. background classification problem, which is highly imbalanced as there are often many more windows with background than foreground objects of interest.

The focal loss, in general, was designed for, and well-suited for classification problems with such `class imbalances`. It is a modification of the classical cross-entropy loss that `puts more focus on harder-to-classify examples, while ignoring the easier examples`.

the cross entropy:

$$L_{c e}\left(y_{\text {true }}, y_{\text {pred }}\right)=-y_{\text {true }} \log \left(p_{\text {pred }}\right)-\left(\mathbf{1}-y_{\text {true }}\right) \log \left(\mathbf{1}-p_{\text {pred }}\right)$$

the focal loss:

$$L_{\text {fo }}\left(y_{\text {true }}, y_{\text {pred }}\right)=-y_{\text {true }} \log \left(p_{\text {pred }}\right) \cdot\left(\mathbf{1}-p_{\text {pred }}\right)^\gamma-\left(\mathbf{1}-y_{\text {true }}\right) \log \left(\mathbf{1}-p_{\text {pred }}\right) \cdot\left(p_{\text {pred }}\right)^\gamma$$


This effect can be seen below, where the focal loss is plotted for different values of γ. For bigger values of γ, well-classified examples (with high probability of y=1) have lower losses, while poorly classified examples have higher losses.

In [64]:
p = np.linspace(0.0001, 1.0, num=100)
for g in [0.0, 0.5, 1, 2, 5, 10]:
    f = - np.power(1 - p, g) * np.log(p)
    plt.plot(p, f, lw=1, label=f"$\gamma$ ={g}")
plt.xlabel("probability of y = 1")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

When ,$\gamma = 0$ the original cross-entropy loss is recovered. As $\gamma$ increases, the part of the curve corresponding to "well-classified" examples becomes longer, reflecting the loss function's focus on poor classification

In [71]:
from scipy.misc import derivative
from scipy.special import expit


def focal_loss(y_true, y_pred, gamma=2.0):
    p = expit(y_pred)
    loss = - (1 - y_true) * np.log(1 - p) * np.power(p, gamma) - y_true * np.log(p) * np.power(1 - p, gamma)
    return loss

In [72]:
#? LightGBM compatible scoring metric
"""
Custom eval function expects a callable with following signatures: func(y_true, y_pred), func(y_true, y_pred, weight) or func(y_true, y_pred, weight, group) and returns (eval_name, eval_result, is_higher_better) or list of (eval_name, eval_result, is_higher_better)
"""


def focal_loss_metric(y_true, y_pred):
    return "focal_loss_metric", np.mean(focal_loss(y_true, y_pred)), False


In [73]:
#? LightGBM compatible objective function
"""
A custom objective function can be provided for the objective parameter. In this case, it should have the signature objective(y_true, y_pred) -> grad, hess, objective(y_true, y_pred, weight) -> grad, hess or objective(y_true, y_pred, weight, group) -> grad, hess
"""


def focal_loss_objective(y_true, y_pred):
    func = lambda z: focal_loss(y_true, z)
    grad = derivative(func, y_pred, n=1, dx=1e-6)
    hess = derivative(func, y_pred, n=2, dx=1e-6)
    return grad, hess

In [86]:
gbm_focal_loss = LGBMClassifier(objective=focal_loss_objective,
                                learning_rate=0.25,
                                n_estimators=20, max_depth=1)
gbm_focal_loss.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=focal_loss_metric)


[1]	valid_0's binary_logloss: 3.38485	valid_0's focal_loss_metric: 0.141808
[2]	valid_0's binary_logloss: 3.01701	valid_0's focal_loss_metric: 0.11333
[3]	valid_0's binary_logloss: 1.10525	valid_0's focal_loss_metric: 0.0952922
[4]	valid_0's binary_logloss: 0.924261	valid_0's focal_loss_metric: 0.0790233
[5]	valid_0's binary_logloss: 0.505147	valid_0's focal_loss_metric: 0.0677255
[6]	valid_0's binary_logloss: 0.682986	valid_0's focal_loss_metric: 0.0606197
[7]	valid_0's binary_logloss: 0.595374	valid_0's focal_loss_metric: 0.0537766
[8]	valid_0's binary_logloss: 0.522227	valid_0's focal_loss_metric: 0.0482069
[9]	valid_0's binary_logloss: 0.215398	valid_0's focal_loss_metric: 0.0459177
[10]	valid_0's binary_logloss: 0.448094	valid_0's focal_loss_metric: 0.0438938
[11]	valid_0's binary_logloss: 0.121162	valid_0's focal_loss_metric: 0.0396912
[12]	valid_0's binary_logloss: 0.351168	valid_0's focal_loss_metric: 0.0373091
[13]	valid_0's binary_logloss: 0.0564804	valid_0's focal_loss_metri

LGBMClassifier(learning_rate=0.25, max_depth=1, n_estimators=20,
               objective=<function focal_loss_objective at 0x00000108801C74C0>)